## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,31.66,86,40,25.32,shower snow
1,1,Faya,SA,18.3851,42.4509,69.91,49,95,3.83,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,35.71,75,20,27.63,few clouds
3,3,Jamestown,US,42.0970,-79.2353,81.95,63,75,17.27,broken clouds
4,4,Namibe,AO,-15.1961,12.1522,63.36,85,1,3.29,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("Enter max temp: "))
min_temp = float(input("Enter min temp: "))

Enter max temp: 60
Enter min temp: 50


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temp_cities_df = city_data_df.loc[(city_data_df["Max Temp"] < max_temp) & (city_data_df["Max Temp"] > min_temp)]
preferred_temp_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Khatanga,RU,71.9667,102.5000,52.56,63,100,8.88,overcast clouds
14,14,Witbank,ZA,-25.8713,29.2332,58.53,44,96,1.50,overcast clouds
16,16,Aklavik,CA,68.2191,-135.0107,57.20,88,75,4.61,shower rain
21,21,Port Alfred,ZA,-33.5906,26.8910,58.21,70,0,2.33,clear sky
37,37,Lebu,CL,-37.6167,-73.6500,53.53,77,91,7.07,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
681,681,Tingi,TZ,-11.3000,35.0333,58.14,73,36,3.56,scattered clouds
700,700,Kamenka,RU,51.3223,42.7678,57.67,85,75,5.03,broken clouds
701,701,Pokosnoye,RU,55.5258,101.0536,54.84,81,32,3.38,scattered clouds
706,706,Teya,RU,60.3778,92.6267,53.17,93,59,2.86,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_temp_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_temp_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Khatanga,RU,52.56,overcast clouds,71.9667,102.5000,
14,Witbank,ZA,58.53,overcast clouds,-25.8713,29.2332,
16,Aklavik,CA,57.20,shower rain,68.2191,-135.0107,
21,Port Alfred,ZA,58.21,clear sky,-33.5906,26.8910,
37,Lebu,CL,53.53,overcast clouds,-37.6167,-73.6500,
38,Ketchikan,US,55.87,overcast clouds,55.3422,-131.6461,
41,Vardo,NO,57.18,clear sky,70.3705,31.1107,
42,Fortuna,US,58.66,broken clouds,40.5982,-124.1573,
43,Nanortalik,GL,52.34,clear sky,60.1432,-45.2371,
69,Port Hardy,CA,55.58,overcast clouds,50.6996,-127.4199,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    results = requests.get(base_url, params=params).json()
     
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = results["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Khatanga,RU,52.56,overcast clouds,71.9667,102.5000,"""Khatanga"""
14,Witbank,ZA,58.53,overcast clouds,-25.8713,29.2332,ANEW Hotel Witbank eMalahleni
16,Aklavik,CA,57.20,shower rain,68.2191,-135.0107,Aklavik Inn
21,Port Alfred,ZA,58.21,clear sky,-33.5906,26.8910,The Halyards Hotel
37,Lebu,CL,53.53,overcast clouds,-37.6167,-73.6500,Hostal Las Lilas


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
